In [3]:
# This cell has the latest set up for AI Platform

from __future__ import print_function
import numpy as np
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import glob
import cv2
from io import BytesIO
from PIL import Image
from numpy import expand_dims
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import *
import warnings
import logging
from IPython.display import clear_output
from collections import Counter
import VisualisationTools as plotting
import pickle

#warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(100)
plot_losses = plotting.PlotLearning()

print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.2.0-dlenv


In [1]:
input_shape = (300)
inputs = Input(shape=input_shape)
print('hello')

t_dense1 = Dense(1024, activation='relu')(inputs)
t_dense2 = Dense(256, activation='relu')(t_dense1)
t_dense3 = Dense(128, activation='relu')(t_dense2)
t_do = Dropout(0.3)(t_dense3)
predictions = Dense(2, activation= 'softmax')(t_do)
model = Model(inputs=inputs, outputs=predictions, name = 'simple_model')

opt = tf.keras.optimizers.Adam(learning_rate= 0.0002, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

NameError: name 'Input' is not defined

In [3]:
psutil.virtual_memory().percent

5.6

In [ ]:
import cv2
import numpy as np
import os
import radialProfile
import glob
from matplotlib import pyplot as plt
import pickle
from scipy.interpolate import griddata

In [2]:
def augment_data(directory, batch):
    '''Prepares train-time augmentation using given training and validations data)
    
    Returns train_data, val_data'''

    datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=True,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=True,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            )

    # Classes give the folders storing the two different categories
    train_data = datagen.flow_from_directory(directory + '/train',
                                             target_size=(224,224), batch_size = batch)
    
    val_data = datagen.flow_from_directory(directory + '/validation',
                                             target_size=(224,224), batch_size = batch)
    
    return train_data, val_data

In [5]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def featurevectorise(img):
    #img = color.rgb2gray(img)
    img = rgb2gray(img)
    h = int(img.shape[1]/3)
    w = int(img.shape[2]/3)
    img = img[0,h:-h,w:-w]

    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    fshift += epsilon


    magnitude_spectrum = 20*np.log(np.abs(fshift))
    # Calculate the azimuthally averaged 1D power spectrum
    psd1D = radialProfile.azimuthalAverage(magnitude_spectrum)
    points = np.linspace(0,N,num=psd1D.size) # coordinates of a
    xi = np.linspace(0,N,num=N) # coordinates for interpolation

    interpolated = griddata(points,psd1D,xi,method='cubic')
    interpolated /= interpolated[0]
    
    return interpolated

In [6]:
batch = 1
train_data, val_data = augment_data('/home/jupyter/all_faces_disk/home/jupyter/forensics_split', batch)

NameError: name 'ImageDataGenerator' is not defined

In [7]:
psutil.virtual_memory()

svmem(total=15774310400, available=14920368128, percent=5.4, used=500813824, free=13108674560, active=1322594304, inactive=682561536, buffers=379981824, cached=1784840192, shared=10887168, slab=577064960)

In [3]:
data= {}
epsilon = 1e-8
N = 300
y = []
error = []


number_iter = 1000
val_length = 100

psd1D_total = np.zeros([number_iter, N])
label_total = np.zeros([number_iter, 2])

X_test = np.zeros([val_length, N])
Y_test = np.zeros([val_length, 2])
cont = 0

for image in train_data:
    psd1D_total[cont,:] = featurevectorise(image[0])             
    label_total[cont,:] = image[1]
    cont+=1
    if cont == number_iter:
        break

NameError: name 'np' is not defined

In [9]:
cont2 = 0
for image in val_data:
    X_test[cont2,:] = featurevectorise(image[0])             
    Y_test[cont2,:] = image[1]
    cont2+=1
    if cont2 == val_length:
        break
print(psd1D_total.shape)
print(X_test.shape)
print(Y_test.shape)

(1000, 300)
(100, 300)
(100, 2)


In [2]:
epochs = 10
batch_size = 32
#X_train = psd1D_total
Y_train = label_total
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

NameError: name 'label_total' is not defined